# Kaggle ML Competiton Entry

## Titanic Survivability Prediction

----

April 2022

# 0. Configure environment

## 0.1 Import libraries

In [1]:
import os
import pandas as pd
import seaborn as sns
import csv

## 0.2 Set data path

In [2]:
data_path = "/".join(os.getcwd().split("/")[:-1]) + "/data/raw/"

## 0.3 Load data

In [3]:
titanic = pd.read_csv(f"{data_path}train.csv")

In [20]:
validation = pd.read_csv(f"{data_path}test.csv")

In [21]:
validation.describe()

,PassengerId,Pclass,Age,SibSp,Parch,Fare
count,418.000000,418.000000,332.000000,418.000000,418.000000,417.000000
mean,1100.500000,2.265550,30.272590,0.447368,0.392344,35.627188
std,120.810458,0.841838,14.181209,0.896760,0.981429,55.907576
min,892.000000,1.000000,0.170000,0.000000,0.000000,0.000000
25%,996.250000,1.000000,21.000000,0.000000,0.000000,7.895800
50%,1100.500000,3.000000,27.000000,0.000000,0.000000,14.454200
75%,1204.750000,3.000000,39.000000,1.000000,0.000000,31.500000
max,1309.000000,3.000000,76.000000,8.000000,9.000000,512.329200


# 1. Data Exploration

----

## 1.1 Data fields

`PassengerId` > Passenger ID

`Survived` > 1 if the passenger survived, 0 if they did not

`Pclass` > Ticket class (1 = 1st, 2 = 2nd, 3 = 3rd)

`Name` > Name of passenger

`Sex` > Sex of passenger

`Age` > Age of passenger

`SibSp` > Number of siblings / spouses aboard the Titanic

`Parch` > Number of parents / children aboard the Titanic

`Ticket` > Ticket Number

`Fare` > Passenger fare

`Cabin` > Cabin number

`Embarked` > Port of embarkation (C = Cherbourg, Q = Queenstown, S = Southampton)


## 1.2 Variable exploration

In [19]:
titanic.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [10]:
titanic.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


# 2. Data preparation

----

## 2.1 Drop unused columns

In [8]:
titanic = titanic.drop(columns = ["Name", "Ticket"])

## 2.2 NaN value handling

In [9]:
# Cabin column
titanic['Cabin'] = titanic['Cabin'].fillna("NONE")

In [19]:
# Age column -- fill all NA values with the average age of all passengers
titanic['Age'] = titanic['Age'].fillna(titanic["Age"].mean())

# 3. Feature engineering

----

## 3.1 Utility functions

## 3.2 Dummy variables

Given the nature and makeup of the data, we will next create dummy variables for:

- Pclass
- Sex
- Cabin (specifically the level)
- Embarked